<a href="https://colab.research.google.com/github/rjlutz/3DPrinting/blob/master/PUWeatherHistoric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup imports and initial variables

We will be calculating the **total rainfall** recorded by the Davis weather instruments within a specific time window at Piedmont University.

We should be able to run this from a smartphone or a browser. The only required inputs are the beginning time and ending time for a time period of interest.

In [2]:
import requests
from datetime import datetime
import time
import json

api_key = "c5hl3euhqgih7xk5yonp6uch8apbojmb"            # API details
api_secret = ""                                         #

# Classes to hold the data

In [3]:
class WeatherData:
  def __init__(self, json_string):
    try:
      data_dict = json.loads(json_string)
      self.station_id_uuid = data_dict["station_id_uuid"]
      self.sensors = [Sensor(**sensor_data) for sensor_data in data_dict["sensors"]]
      self.generated_at = data_dict["generated_at"]
    except json.JSONDecodeError as e:
      print(f"Error parsing JSON: {e}")
      raise

class Sensor:
  def __init__(self, lsid, data, sensor_type, data_structure_type):
    self.lsid = lsid
    self.data = data  # List of data points (dictionaries)
    self.sensor_type = sensor_type

    self.data_structure_type = data_structure_type

# Utility function to convert user's start/end to 'Unix time'.
Unix time is required by the Weatherlink service and is the number of seconds since the Unix Epoch, aka 1/1/1070 GMT.

In [4]:
def get_unix_timestamp(time_string):
  date_time = datetime.strptime(time_string, "%Y/%m/%d %H:%M:%S") # Convert time to datetime object
  return int(date_time.timestamp())   # Get and return Unix timestamp in seconds

# Two more functions

The first will call the Weatherlink service at:

    https://api.weatherlink.com/v2/historic

with query parameters like:

    api-key = our api_key
    start-timestamp = unix_time_to_begin
    end-timestamp = unix_time_to_end

We also provide a http header with the api secret


The second function tallies the rainfall in inches across a time window. Weatherlink limits us from requesting more that 24 hours in a call, so we will call this fir every 24 period in the requested window.


In [5]:
def get_weather_data(station_id, ts):

  url = f"https://api.weatherlink.com/v2/historic/{station_id}?api-key={api_key}&start-timestamp={ts[0]}&end-timestamp={ts[1]}"
  headers = {"X-Api-Secret": api_secret}

  try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    weather_data = WeatherData(response.text)
    return weather_data

  except requests.exceptions.RequestException as e:
    if e.response.status_code == 400:
        print(e.response.text)
        exit()
    print(f"Error: An error occurred while making the request: {e}")
    return None

def calculate_total_rain(weather_data):

  total_rain = 0

  for sensor in weather_data.sensors:
      if sensor.sensor_type != 37:
          continue
      for datum in sensor.data:
        total_rain += float(datum['rainfall_in'])
        # print(f"ts:{datum['ts']} rainfall_in: {datum['rainfall_in']}")

  return total_rain

# Finally! The calculation!

We use the imports, varaiables, classes and functions defined above

In [7]:
wetlands_station_id = 72242                      # Set station ID
                                                 # rainfall from 2024/05/04 00:00:00 to 2024/05/05 00:00:00
default_start = "2024/05/04 00:13:16"            # inches
default_end = "2024/05/20 00:13:17"              # Prompt the user for the beginning and end times
s_prompt = f"Enter start time ({default_start}): "
e_prompt = f"Enter end time ({default_end}): "
start_string = input(s_prompt) or default_start
end_string = input(e_prompt) or default_end

ts = []                                          # ts has begin/ending timestamps from the user
ts.append(get_unix_timestamp(start_string))
ts.append(get_unix_timestamp(end_string))

begin = ts[0]
total_rainfall = 0
print()
while ts[1] - begin >= 0:
  end = ts[1] if ts[1] - begin < 86400 else begin+86399
  b_formatted_time = time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(begin))
  e_formatted_time = time.strftime("%Y/%m/%d %H:%M:%S", time.localtime(end))

  weather_data = get_weather_data(wetlands_station_id, [begin, begin+86400])
  this_rainfall = calculate_total_rain(weather_data)
  total_rainfall += this_rainfall
  print(f'Rainfall from {b_formatted_time} to {e_formatted_time}: {this_rainfall:.2f} (accumulated {total_rainfall:.2f})')
  begin = end+1


print (f'\nTotal rainfall: {total_rainfall:.2f}')

Enter start time (2024/05/04 00:13:16): 2024/01/01 00:01:00
Enter end time (2024/05/20 00:13:17): 2024/03/17 11:59:30

Rainfall from 2024/01/01 00:01:00 to 2024/01/02 00:00:59: 0.00 (accumulated 0.00)
Rainfall from 2024/01/02 00:01:00 to 2024/01/03 00:00:59: 0.00 (accumulated 0.00)
Rainfall from 2024/01/03 00:01:00 to 2024/01/04 00:00:59: 0.00 (accumulated 0.00)
Rainfall from 2024/01/04 00:01:00 to 2024/01/05 00:00:59: 0.00 (accumulated 0.00)
Rainfall from 2024/01/05 00:01:00 to 2024/01/06 00:00:59: 0.00 (accumulated 0.00)
Rainfall from 2024/01/06 00:01:00 to 2024/01/07 00:00:59: 1.34 (accumulated 1.34)
Rainfall from 2024/01/07 00:01:00 to 2024/01/08 00:00:59: 0.00 (accumulated 1.34)
Rainfall from 2024/01/08 00:01:00 to 2024/01/09 00:00:59: 0.00 (accumulated 1.34)
Rainfall from 2024/01/09 00:01:00 to 2024/01/10 00:00:59: 3.76 (accumulated 5.10)
Rainfall from 2024/01/10 00:01:00 to 2024/01/11 00:00:59: 0.02 (accumulated 5.12)
Rainfall from 2024/01/11 00:01:00 to 2024/01/12 00:00:59: 0.0